<a href="https://colab.research.google.com/github/stefanbringuier/randomonium/blob/main/notebooks/QuAcc_Phonon_QE_Example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Prepared by [Stefan Bringuier](https://stefanbringuier.info)**

**Caution:** This is just for demo purposes.

In [1]:
%%capture
!pip install -q condacolab

In [2]:
%%capture
import condacolab
condacolab.install()

In [3]:
%%capture
# Install development version of ASE
! pip install --force-reinstall --no-deps https://gitlab.com/ase/ase/-/archive/master/ase-master.zip
! pip install quacc

In [4]:
%%capture
! conda install -c conda-forge qe

In [5]:
%%capture
! pip install quacc[dask]

In [6]:
%%capture
! wget "https://archive.materialscloud.org/record/file?filename=SSSP_1.3.0_PBE_efficiency.tar.gz&record_id=1732" -O SSSP_1.3.0_PBE_efficiency.tar.gz
! mkdir -p pseudopotentials
! tar -xzvf /content/SSSP_1.3.0_PBE_efficiency.tar.gz -C pseudopotentials

In [7]:
from quacc import SETTINGS

SETTINGS.ESPRESSO_PSEUDO = "/content/pseudopotentials"
SETTINGS.ESPRESSO_BIN_DIR = "/usr/local/bin"

In [10]:
from ase.build import bulk
from quacc.recipes.espresso.core import relax_job
from quacc.recipes.espresso.phonons import phonon_job
from quacc.recipes.espresso.phonons import grid_phonon_flow

In [12]:
atoms = bulk("Cu")

job_1 = relax_job(atoms,
                occupations='smearing',
                smearing='gaussian',
                degauss=0.02,
                kpts=(6,6,6))
job_1.compute()

{'atoms': Atoms(symbols='Cu', pbc=True, cell=[[0.0, 1.8050006441977142, 1.8050006441977142], [1.8050006441977142, 0.0, 1.8050006441977142], [1.8050006441977142, 1.8050006441977142, 0.0]], initial_magmoms=...),
 'atoms_info': {'_id': '2b07562e721a386fbaad817f20375192'},
 'builder_meta': {'build_date': datetime.datetime(2024, 2, 7, 19, 19, 1, 253500),
  'emmet_version': '0.77.1',
  'pymatgen_version': '2024.1.27'},
 'chemsys': 'Cu',
 'composition': Composition('Cu1'),
 'composition_reduced': Composition('Cu1'),
 'density': 8.971710194693667,
 'density_atomic': 11.761482842878042,
 'dir_name': '/content/quacc-2024-02-07-19-18-32-741474-26060',
 'elements': [Element Cu],
 'formula_anonymous': 'A',
 'formula_pretty': 'Cu',
 'input_atoms': {'atoms': Atoms(symbols='Cu', pbc=True, cell=[[0.0, 1.805, 1.805], [1.805, 0.0, 1.805], [1.805, 1.805, 0.0]], calculator=Espresso(...)),
  'atoms_info': {},
  'builder_meta': {'build_date': datetime.datetime(2024, 2, 7, 19, 19, 1, 240606),
   'emmet_versio

**Note** On Google colab you have to call the `compute` method.

In [13]:
job_2 = phonon_job(
      job_1['dir_name'],
      parallel_info=None,
      test_run=False,
      occupations='smearing',
      smearing='gaussian',
      degauss=0.02,
      kpts=(6,6,6)
)
job_2.compute()

{'dir_name': '/content/quacc-2024-02-07-19-19-57-868455-57664',
 'name': 'ph.x Phonon',
 'nid': 'dc64836c3b3e',
 'parameters': {'degauss': 0.02,
  'input_data': {'inputph': {'tr2_ph': 1e-12, 'alpha_mix(1)': 0.1, 'verbosity': 'high'}},
  'kpts': (6, 6, 6),
  'occupations': 'smearing',
  'qpts': (0, 0, 0),
  'smearing': 'gaussian'},
 'quacc_version': '0.6.4',
 'results': {(0.0,
   0.0,
   0.0): {'atoms': Atoms(symbols='Cu', pbc=True, cell=[[0.0, 1.8049827756085057, 1.8049827756085057], [1.8049827756085057, 0.0, 1.8049827756085057], [1.8049827756085057, 1.8049827756085057, 0.0]]), 'eqpoints': array([[0., 0., 0.]]), 'freqs': array([0.32597, 0.32597, 0.32597]), 'kpoints': {1: array([[ 0.       ,  0.       ,  0.       ,  0.0092593],
           [ 0.1178511,  0.1178511, -0.1178511,  0.0740741],
           [ 0.2357023,  0.2357023, -0.2357023,  0.0740741],
           [-0.3535534, -0.3535534,  0.3535534,  0.037037 ],
           [ 0.2357023,  0.       ,  0.       ,  0.0555556],
           [ 0.3535

**The job below will crash on Google colab instances**

In [ ]:
qe_params = {'occupations':'smearing',
             'smearing':'gaussian',
             'degauss': 0.02,
             'kpts': (6,6,6),
             'koffset': (0, 0, 0),
}
ph_flow = grid_phonon_flow(
    atoms,
    nblocks=0,
    job_decorators=None,
    job_params={'relax_job':qe_params,
                'ph_job':qe_params}
)
ph_flow.compute()